In [ ]:
# example of a multi-output regression problem
from sklearn.datasets import make_regression
# create dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)
# summarize shape
print(X.shape, y.shape)

In [ ]:
y

In [ ]:
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [34]:
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results

In [36]:
a = evaluate_model

In [ ]:
evaluate_model(X, y)

In [27]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Embedding, Bidirectional, Dense, LSTM, Input, Flatten
from keras.initializers import TruncatedNormal
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import argparse

In [2]:
add_token_data = './data/wikitext-2/DVA_dataset.csv'
train_data_file = './data/train0.csv'
val_data_file = './data/val0.csv'
test_data_file = './data/test0.csv'
lstm_hidden_size = 200
Embedding_size = 768

In [3]:
train_data = pd.read_csv(train_data_file,header=None)
token_data = pd.read_csv(add_token_data,header=None)
val_data = pd.read_csv(val_data_file,header=None)
test_data = pd.read_csv(test_data_file,header=None)

In [4]:
token_split = [([w.split('(')[:-1][0] for w in i.split(' ')]) for i in token_data[1]]
test_split = [([w.split('(')[:-1][0] for w in i.split(' ')]) for i in test_data[1]][:100]
train_split = [([w.split('(')[:-1][0] for w in i.split(' ')]) for i in train_data[1][:100]]
val_split = [([w.split('(')[:-1][0] for w in i.split(' ')]) for i in val_data[1]]

In [5]:
max_len = len(max(token_split, key=len))

In [6]:
tk = Tokenizer()
tk.fit_on_texts(token_split)
len(tk.word_index)

1603

In [7]:
train = tk.texts_to_sequences(train_split)
test = tk.texts_to_sequences(test_split)

In [8]:
train = pad_sequences(train, maxlen=max_len, padding='post')
test = pad_sequences(test, maxlen=max_len, padding='post')

In [41]:
train_label = train_data[[3]][:100].values
test_label = test_data[[3]][:100].values

In [42]:
model = Sequential()
model.add(Embedding(len(tk.word_index)+1,  2,  input_length=max_len))
model.add(Bidirectional(LSTM(1, 
                            activation="relu", 
                            return_sequences=True)))
model.add(Bidirectional(LSTM(1, 
                             activation="relu")))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 2)             3208      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 2)             32        
_________________________________________________________________
bidirectional_4 (Bidirection (None, 2)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 3,275
Trainable params: 3,275
Non-trainable params: 0
_________________________________________________________________


In [ ]:
input_dim = Input(shape=(max_len,), name='input_dim')
inputs = {'input_dim': input_dim}

x = Embedding(len(tk.word_index)+1, 8, input_length=max_len)(input_dim)
x = Bidirectional(LSTM(1, activation="relu", return_sequences=True))(x)
x = Bidirectional(LSTM(1, activation="relu"))(x)

valence = Dense(units=1, name='valence')(x)
arousal = Dense(units=1, name='arousal')(x)
outputs = {'valence': valence, 'arousal': arousal}

model = Model(inputs=input_dim, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()

In [43]:
opt = Adam(learning_rate=1)
model.compile(loss='mse', optimizer='adam')

In [79]:
checkpointer = ModelCheckpoint(filepath="./a/weights.hdf5", save_best_only=True)
history = model.fit(train, train_label, validation_data=[train,train_label], callbacks=[checkpointer],epochs=10)

Train on 100 samples, validate on 100 samples
Epoch 1/10
100/100 [==============================] - 0s 2ms/step - loss: 9.5278 - val_loss: 9.5103


OSError: Unable to create file (unable to open file: name = './a/weights.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [30]:
model.evaluate(train, train_label)

100/100 [==============================] - 0s 244us/step


0.18817562818527223

array([3.88888889, 3.375     , 4.66666667, 3.5       , 2.88888889,
       3.        , 3.33333333, 3.375     , 2.9       , 3.        ,
       3.        , 3.14285714, 3.71428571, 3.4       , 3.1       ,
       3.66666667, 2.75      , 3.        , 3.55555556, 3.        ,
       2.625     , 3.        , 3.66666667, 3.55555556, 2.66666667,
       3.        , 3.5       , 3.375     , 3.88888889, 3.6       ,
       2.55555556, 3.        , 3.44444444, 3.5       , 3.625     ,
       3.9       , 3.1       , 3.        , 3.66666667, 2.9       ,
       3.625     , 3.55555556, 2.6       , 2.88888889, 3.33333333,
       4.        , 3.        , 3.11111111, 3.7       , 2.88888889,
       3.75      , 2.875     , 3.5       , 2.625     , 3.28571429,
       3.5       , 3.11111111, 3.        , 3.55555556, 3.        ,
       3.44444444, 3.        , 3.375     , 3.        , 3.55555556,
       2.625     , 3.        , 3.5       , 2.625     , 3.66666667,
       2.75      , 3.44444444, 2.5       , 3.44444444, 3.5    

In [60]:
pd.DataFrame({'b':a,'a':a})

,b,a
0,0.162087,0.162087
1,0.162087,0.162087
2,0.162087,0.162087
3,0.162087,0.162087
4,0.162087,0.162087
5,0.162087,0.162087
6,0.162087,0.162087
7,0.162087,0.162087
8,0.162087,0.162087
9,0.162087,0.162087


In [28]:
hist_df = pd.DataFrame(history.history) 

In [33]:
hist_df

,val_loss,loss
0,0.228147,0.228195
1,0.228184,0.227991
2,0.228265,0.228132
3,0.228172,0.228356
4,0.227990,0.228118
5,0.227889,0.227967
6,0.227800,0.227894
7,0.227650,0.227750
8,0.227531,0.227783
9,0.227442,0.227517


In [76]:
with open('max_len.text', 'w') as f:
    f.write(str(4))
    f.close()

In [77]:
with open('max_len.text', 'r') as f:
    max_len = int(f.readline())

In [78]:
max_len

'4'